# <font color='purple'>Práctica Fundamentos de Criptografía</font> 
---
![](https://www.criptotendencias.com/wp-content/uploads/2018/12/cifrado-criptomonedas.jpg)



## <font color="teal"> Ejercicio 1: Cifrado y Descifrado </font>

En este ejercicio vamos cifrar y decifrar textos de la manera que lo haría un cifrador de flujo, es decir, de carácter en carácter. 

Para ello se necesita:
  1. Generar una secuencia super-creciente de números positivos.
  2. Generar las llaves para el cifrador.
  3. Utilizaremos algunas librerías ya hechas para  binarizar el mensaje.

In [5]:
import numpy as np
import AritmeticaModular as am
from random import randint

"""
    Genera una secuencia super-creciente de longitud k.
    
    Una secuencia super-creciente tiene k elementos donde
    
                    a_i > sum(a_0,...,a_(i - 1))
                    
    Un ejemplo: (1, 2, 4, 8, 16)
    
    k: longitud del bloque del mensaje
"""
def gen_super_creciente(k):
    
    # generar el primer elemento de la secuencia
    sequence = [0]*k
    sequence[0] = randint(1, k)
    
    for i in range(1, k):
        sequence[i] = sum(sequence[:i]) + randint(sequence[i-1], sequence[i-1]*3)
            
    return np.array(sequence, dtype = np.uint32)

gen_super_creciente(8)

array([    6,    15,    63,   192,   801,  2324, 10030, 35940],
      dtype=uint32)

In [6]:
"""
    Genere las claves para el cifrador.
    
    Parametros:
        block_length: longitud del bloque del mensaje
    Return:
        public_key: secuencia modificada
        private_key: secuencia super-creciente original, n y u
"""

def generate_keys(block_length):
    # generate a sequence
    sequence = gen_super_creciente(block_length)
    
    n = sum(sequence) + sequence[randint(0,block_length-1)]*2
    
    found_u = False
    
    while not found_u:
        
        u = randint(1, n)
        
        if am.ext_euclides(n,u)[0] == 1: 
            found_u = True
            
    public_sequence = np.array(list(map(lambda a_i: (u*a_i) % n, sequence)), dtype = np.uint32)
    
    return public_sequence, (sequence, n, u)

key_pub, key = generate_keys(8)
print("Llave pública:", key_pub)
print("Llave privada:", key[0], key[1], key[2])

Llave pública: [  569  8298  4552  2750 12328 12516  9950 11643]
Llave privada: [   2    5   16   58  140  624 2210 6276] 13751 7160


Una vez generadas nuestras llaves, podemos pasar a realizar el cifrado del mensaje. Para cifrar el mensaje tendremos que pasar el string a binario, y para ello, usaremos las funciones *str_to_binlist* y *bin_to_str*. Una vez realizado esto, podremos pasar a a cifrar el mensaje en bloques de tamaño $k$.

In [121]:
from CifradoFlujo import bin_to_str, str_to_binlist

"""
                            Cifrador
    
    Esta función toma un mensaje y la llave
    pública y encripta el mensaje realizando:
            ___n
            \                 a_i * e_i
            /__ 0
            
    donde a_i son los bits del mensaje y e_i
    los elementos de la llave pública.
    
    Parametros:
        - message: un str que contiene el mensaje simple.
        - key_pub: llave pública
"""
def knapsack_cypher(message, key_pub):
    # toma el tamaño del bloque
    block_size = len(key_pub)
    # transforma el mensaje a una lista binaria
    binary_message = str_to_binlist(message)
    # comprueba si el mensaje es divisible por el tamaño del bloque
    # si no, se agregan ceros al final
    while len(binary_message) % block_size != 0:
        binary_message.append(0)
    
    cypher, binary_message = [], np.array(binary_message)
     # cypher text
    for i in range(0,len(binary_message), block_size):
        cypher.append(np.sum(key_pub[binary_message[i:i + block_size] == 1]))
        
    return cypher


encrypted = knapsack_cypher("Pon tu nombre aqui", key_pub)

print("Texto encriptado:", encrypted)

Texto encriptado: [11048, 59287, 47644, 4552, 28116, 39759, 4552, 47644, 59287, 49337, 22800, 25550, 37009, 4552, 24493, 27243, 39759, 36821]


In [122]:
from AritmeticaModular import inverse
"""
                                        Descifrador
                        
Para descifrar un mensaje cifrado con este algoritmo,
es necesario calcular v como u^{-1} mod n usando el
algoritmo de Euclides.

Luego, el mensaje se descifra:
            _
        b = vb* mod n
            _     ___  
            = v \       e_i a*_i (mod n)
                   /__
            _          ___
            = (uv) \    e_i a_i
                        /__
            _   ___           
            =   \        e_i a_i   (mod n)
                 /__
                 
Parametros:
    - message: lista de enteros que representan el mensaje.
    - key: llave privada.
"""

def knapsack_decypher(message, key):
    # desplegar la clave privada y calcula
    # el inverso de u mod n
    p_key, n, u = key
    v =  inverse(u, n)
    # Concatenar listas en Python es muy simple si usa el operador + y dos listas.
    # Esa es la razón para hacer decypher_message una lista vacía
    decypher_message = []
    # iniciar descifrador
    for b in message:
        # b_ = vb mod n
        b_, aux = (b * v) % n, 0
        # iniciar el algoritmo greedy (codicioso)
        greedy_result = set()
        for i in range(1, len(p_key) + 1):
            if aux + p_key[-i] <= b_:
                aux += p_key[-i]
                greedy_result.add(p_key[-i])
                if aux == b_:
                    break
        # restaura y agrega el mensaje
        decypher_message += list(map(lambda x: 1 if x in greedy_result else 0, p_key))
        # devuelve el texto simple
    return bin_to_str(decypher_message)


knapsack_decypher(encrypted, key)

'Pon tu nombre aqui'

## Actividad a relizar:

El alumno debe leer el archivo de preguntas cifradas y hacer los cambios necesarios en la celda siguiente para conseguir descifrar las preguntas y contestarlas. Estas están cifradas mediante las claves:

Llave pública: [  569  8298  4552  2750 12328 12516  9950 11643]

Llave privada: [   2    5   16   58  140  624 2210 6276] 13751 7160

Adicionalmente el alumno debe escribir sus respuestas sobre el mismo archivo txt y modificar la celda posterior para cifrar sus respuestas bajo la llave pública que aparece arriba.

In [ ]:
key = []
lines = []
with open("Cambiame", 'r') as f:
       for i in range(10):
         lines.append(eval(f.readline()))

with open("Preguntas_descifradas.txt", 'w') as f:
  for i in range(10):
    f.write(str("Aqui se decifra el archivo")) 

In [ ]:
key_pub = []
with open("Respuestas_cifradas.txt", '') as f:
  for i in range(10):
    f.write(str("Aquí se cifran tus respuestas")+ "\n")

## <font color="navy"> Ejercicio 2: RSA </font>

En este ejercicio se pide implementar un sistema de firma digital y verificación de la firma mediante el algoritmo RSA.

Se deben realizar tres tareas:
  1. Generación de claves
  2. Generación de firma
  3. Verificación de firma

Para la generación de firma, se le introducirá un mensaje a cifrar (archivo) y el archivo con la clave (privada), y deberá generar una firma, que se guardará en un archivo de texto.

Puesto que lo que realmente se firma no es el mensaje, sino un resumen del mensaje, hay que generar un resumen de dicho mensaje, Para esto emplearemos la función SHA1 (se pueden añadir otras funciones resumen).

In [9]:
from random import getrandbits
from hashlib import sha1, sha256, sha512

def get_hash(file, hash_f):
    # calcular el hexhash de todo el archivo
    # y convertirlo a base 10
    if hash_f == 1:
        file_hash = int(sha1(file.read()).hexdigest(), 16)
    elif hash_f == 2:
        file_hash = int(sha256(file.read()).hexdigest(), 16)
    elif hash_f == 3:
        file_hash = int(sha512(file.read()).hexdigest(), 16)
    else:
        raise ValueError("No es una opción adecuada para el algoritmo de resumen.")
        
    return file_hash

In [13]:
"""
                                    Search prime
                        
     Busca un número primo mayor que un número.
     Este primo, dependiendo de los parámetros
     tendrá la propiedad de que p y (p - 1) // 2 sean primos.
     
     Parametros:
         - n: número inicial.
         - mid_prime: busca un primo de manera que (p - 1) // 2 sea también primo.
"""

# busca un primo de manera que (p - 1) // 2 sea también primo (probablemente)
def search_prime(n, mid_prime = True):
    p = n if n % 2 != 0 else n + 1
    if mid_prime:
        while not (am.miller_rabin_test(p) and
                   am.miller_rabin_test((p -1)//2)):
                p +=2
    else:
        while not am.miller_rabin_test(p):
            p += 2
    return p

"""
                        Generador de claves RSA
                        
    Esta función crea un par de claves RSA
    con una longitud fija. Primero busca dos primos
    p y q, para obtener n.
    
    Entonces busca φ y e tal que gcd(e, φ(n)) = 1, y 
    computa d como e^{-1} mod φ(n)
    
    La llave privada es n y d, y la 
    llave pública es n y e.
    
    Parametros:
            - length: longitud en bits de la clave.
"""

def generate_RSA_keys(length = 1024):
    # Calcula los primos p y q
    p = search_prime(getrandbits(length//2), False)
    q = search_prime(getrandbits(length//2), False)
    # Calcula φ(n), n y a corrige e
    φ, n = (p - 1) * (q - 1), p * q
    e = randint(n - φ, φ)
    while am.ext_euclides(e, φ)[0] != 1:
        e += 1
    
    # Calcula la llave privada
    
    d = am.inverse(e, φ) % φ

    # Escriba la llave pública y la llave privada en archivos
    with open("RSA_KEY.pub", 'w') as f:
        f.write(str(n) + "\n" + str(e))
    
    with open("RSA_KEY", 'w') as f:
        f.write(str(n) + "\n" + str(d))

        
generate_RSA_keys(2048)

In [14]:
"""
                            RSA_sign_document
                    
    Esta función toma un archivo y lo firma con
    su clave RSA privada. La función calcula
    el resumen del archivo con sha1, sha256 o
    sha512 y firma este resumen con la clave privada.
    
    Luego, la firma se escribe en un archivo.
    
    Parametros:
        - file: ruta al archivo.
        - key: ruta al archivo con la clave privada
        - hash_f: opción de número entero para el algoritmo de resumen
"""

def RSA_sign_document(file, key, hash_f = 1):
    with open(file, 'rb') as f:
        file_hash = get_hash(f, hash_f)
            
    with open(key, 'r') as f:
        n = int(f.readline())
        d = int(f.readline())
    
    # firma el documento sign(m) = h(m)^d mod n
    sign = am.big_pow(file_hash, d, n)
    # escribe la firma
    with open("sign_of_" + file, 'w') as f:
        f.write(str(sign))
    # la devolución es solo para ver que la función funciona bien
    return sign

RSA_sign_document("Cielito_lindo.txt", "RSA_KEY", 3)

665459917452247328771342180910363450882540397188940263483921753139599639958044561679357939561984602018128644021897588343753324473955049725717361471855521790831500201968929879465618347583150965028439721021817771207678931817891008527494896208300041754319334963022436041232388738831313800864724202417495751450259165637575353483127666110856770121948471050604212111050053112704728960929823117186740509884599823830503772365435031211868036594116912269378734461657043249428022322291961224749869177702372505912852991064034110269945252564552204304943757516884122059176471634769668462744934252363776665856547438337286072593271

Para la verificaión de la firma, se introduce el mensaje (archivo) que se ha firmado, un archivo con la firma (con el mismo formato que el generado en el apartado anterior) y un archivo con la clave (pública). Deberá responder si la firma es o no válida.

In [15]:
"""
                            RSA_check_sign
                    
    Esta función toma un archivo y comprueba si la firma
    es correcta con la clave pública. Como en la función de
    la firma, esta función calcula el resumen del archivo 
    con sha1, sha256 o sha512, luego calcula sign^e mod n 
    y comprueba si los valores hash son iguales.
    
    Parametros:
        - sign_document: ruta a la firma.
        - document: ruta al archivo del documento.
        - key: ruta al archivo con la clave privada
        - hash_f: opción de número entero para el algoritmo de resumen
"""

def RSA_check_sign(sign_document, document, public_key, hash_f = 1):
    # obtener la firma
    with open(sign_document, 'r') as f:
        sign = int(f.read())
    
    # obtener la clave pública
    with open(public_key, 'r') as f:
        n = int(f.readline())
        e = int(f.readline())
    
    # obtener el hash del documento
    with open(document, 'rb') as f:
        h_m = get_hash(f, hash_f) % n
        
    sign = am.big_pow(sign, e, n)
    return h_m == sign
    

RSA_check_sign("sign_of_Cielito_lindo.txt", "Cielito_lindo.txt", "RSA_KEY.pub", 3)

True

Una vez obtenida la firma, para comprobar que la firma es correcta, el emisor y el receptor deben ponerse de acuerdo para el algoritmo que realiza el hash del mensaje, y una vez establecido, sólo hay que leer el documento que tiene la firma y obtener el hash original haciendo $h(m) = sign^e \bmod n$. Una vez obtenido, hacemos el hash al fichero firmado y comprobamos si el hash que hemos calculado y hash obtenido de la firma coinciden. En caso de que sea así, la firma es correcta y en caso contrario, el fichero no pertenece al propietario que lo firmó.